In [ ]:
import math
import re

import openrouteservice as ors
import pandas as pd
import plotly.express as px

from geopy.geocoders import Nominatim

In [91]:
# Set your city manually here
city = "Fort Worth, TX"  # Change this as needed

In [92]:
# Define the pattern for the city (case insensitive)
city_pattern = re.compile(re.escape(city), re.IGNORECASE)

# Open and read the file
with open("listings.txt", "r") as file:
    data = file.read()

# Find all lines that contain the city pattern
matches = [line for line in data.splitlines() if city_pattern.search(line)]

# Organize data by address (set to store unique addresses)
unique_addresses = []
address_prices = {}  # To store prices for each address

# Go through each match and process data
for match in matches:
    # If there's a vertical bar '|', get everything after it (excluding space)
    if '|' in match:
        match = match.split('|', 1)[1].lstrip()  # Extract after the bar, remove leading spaces

    # Add to list of unique addresses
    if match not in unique_addresses:
        unique_addresses.append(match)
        address_prices[match] = set()  # Use a set to prevent duplicate prices

# Iterate through the file again, this time collecting prices for each address
current_address = None

# Extract prices from the data
for line in data.splitlines():
    # Check if the line contains any of the unique addresses
    for address in unique_addresses:
        if address in line:
            current_address = address  # Set current address when we find it
            break

    if current_address:  # If we've set a current address, check for prices
        price_matches = re.findall(r'\$(\d{1,3}(?:,\d{3})*)', line)  # Match prices
        
        if price_matches:
            for price in price_matches:
                address_prices[current_address].add(price)  # Use set to avoid duplicates

# Now, convert the address_prices dictionary into a DataFrame
rows = []
columns = ['Address', 'Price']

# Prepare rows for the DataFrame
for address, prices in address_prices.items():
    for price in prices:
        rows.append([address, price])

df = pd.DataFrame(rows, columns=columns)

In [93]:
pd.set_option('display.max_colwidth', None)

df.head()

,Address,Price
0,"6351 Hulen Bend Blvd, Fort Worth, TX",795
1,"6351 Hulen Bend Blvd, Fort Worth, TX","1,247"
2,"6751 Westcreek Dr, Fort Worth, TX",799
3,"3500 Renzel Blvd, Fort Worth, TX","1,199"
4,"3500 Renzel Blvd, Fort Worth, TX",845


In [94]:
# OpenRouteService Client
client = ors.Client(key='5b3ce3597851110001cf624817e35411a1444fc7901b0872949c4b86')

# Initialize Geolocator for address geocoding
geolocator = Nominatim(user_agent="address_optimizer")

# Fixed intermediate addresses (coordinates) that will be used for every route
intermediate_coords = [
    (32.74840963982297, -97.32497439905646),  # Address 1
    (32.747705801640954, -97.35808003618311),  # Address 2
    (32.7515678225884, -97.35133168289146)     # Address 3
]

# Function to calculate the centroid of the intermediate locations
def calculate_centroid(coords):
    latitudes = [coord[0] for coord in coords]
    longitudes = [coord[1] for coord in coords]
    
    centroid_lat = sum(latitudes) / len(coords)
    centroid_lon = sum(longitudes) / len(coords)
    
    return centroid_lat, centroid_lon

# Calculate the centroid of the intermediate locations
centroid_lat, centroid_lon = calculate_centroid(intermediate_coords)

# List to store results for each address
walk_times = []

# Iterate over each address in the DataFrame
for index, row in df.iterrows():
    # Geocode the address for this row
    location = geolocator.geocode(row['Address'])
    
    if location:
        address_lat = location.latitude
        address_lon = location.longitude
        
        # Use ORS Directions API to get walking time from the address to the centroid
        route = client.directions(
            coordinates=[(address_lon, address_lat), (centroid_lon, centroid_lat)], 
            profile='foot-walking', 
            format='geojson'
        )
        
        # Extract the total walking time (in seconds)
        if route['features']:
            walking_time_seconds = route['features'][0]['properties']['segments'][0]['duration']
            walking_time_minutes = walking_time_seconds / 60  # Convert to minutes
        else:
            walking_time_minutes = None  # In case there is an issue with the routing API
        
        # Store the result
        walk_times.append({
            'Address': row['Address'],
            'Price': row['Price'],
            'Walking Time to Centroid (minutes)': walking_time_minutes
        })

# Convert the results into a DataFrame for better readability
walk_times_df = pd.DataFrame(walk_times)

In [116]:
# Remove commas from the 'Price' column and convert it to numeric
walk_times_df_sorted['Price'] = walk_times_df_sorted['Price'].str.replace(',', '').astype(float)

# Sort the DataFrame based on 'Walking Time to Centroid (minutes)'
walk_times_df_sorted = walk_times_df_sorted.sort_values(by='Walking Time to Centroid (minutes)', ascending=True)

# Save the sorted DataFrame to a new CSV file
walk_times_df_sorted.to_csv('walk_times_sorted.csv', index=False)

# Display the DataFrame
walk_times_df_sorted

,Address,Price,Walking Time to Centroid (minutes)
31,"1001 W 7th St, Fort Worth, TX",1190.0,16.696667
30,"1012 Burnett St, Fort Worth, TX",1184.0,19.608333
27,"929 Norwood St, Fort Worth, TX",1150.0,27.528333
29,"3125 Sondra Dr, Fort Worth, TX",1182.0,34.093333
28,"3125 Sondra Dr, Fort Worth, TX",1175.0,34.093333
22,"555 Elm St, Fort Worth, TX",1106.0,34.726667
36,"320 E Broadway Ave, Fort Worth, TX",1234.0,38.905000
37,"336 Oakhurst Scenic Dr, Fort Worth, TX",1250.0,64.921667
11,"4000 S Hulen St, Fort Worth, TX",998.0,79.593333
38,"3224 Sandage Ave, Fort Worth, TX 76109",1250.0,88.485000


In [117]:
walk_times_df_sorted = pd.read_csv('walk_times_sorted.csv')

In [ ]:
# Ensure 'Price' is numeric (in case it wasn't recognized correctly)
walk_times_df_sorted['Price'] = pd.to_numeric(walk_times_df_sorted['Price'], errors='coerce')

# Initialize a new column to count the number of dominated points
walk_times_df_sorted['Dominated Count'] = 0

# Loop through each point and compare it with all other points to count the dominated points
for i in range(len(walk_times_df_sorted)):
    for j in range(len(walk_times_df_sorted)):
        # If point i dominates point j, increment the dominated count for point i
        if (walk_times_df_sorted.iloc[i]['Price'] < walk_times_df_sorted.iloc[j]['Price'] and
            walk_times_df_sorted.iloc[i]['Walking Time to Centroid (minutes)'] < walk_times_df_sorted.iloc[j]['Walking Time to Centroid (minutes)']):
            walk_times_df_sorted.at[i, 'Dominated Count'] += 1

# Create a new column for the "Pareto front"
walk_times_df_sorted['Pareto Front'] = True
for i in range(1, len(walk_times_df_sorted)):
    if (walk_times_df_sorted.iloc[i]['Price'] >= walk_times_df_sorted.iloc[i-1]['Price'] and 
        walk_times_df_sorted.iloc[i]['Walking Time to Centroid (minutes)'] >= walk_times_df_sorted.iloc[i-1]['Walking Time to Centroid (minutes)']):
        walk_times_df_sorted.at[i, 'Pareto Front'] = False

# Filter out the Pareto front
pareto_front = walk_times_df_sorted[walk_times_df_sorted['Pareto Front']]

# Create an interactive scatter plot using Plotly, color the points based on "Dominated Count"
fig = px.scatter(
    pareto_front, 
    x='Price', 
    y='Walking Time to Centroid (minutes)', 
    hover_name='Address',  # This shows the address only on hover
    title='Pareto Front: Price vs Walking Time',
    labels={'Price': 'Price ($)', 'Walking Time to Centroid (minutes)': 'Walking Time (minutes)'},
    color='Dominated Count',  # Color based on how many points the point dominates
    color_continuous_scale='Viridis',  # Optional: you can change the color scale
    template='plotly_dark'
)

# Adding a small margin to the axis range to prevent points from going off the edges
x_margin = (pareto_front['Price'].max() - pareto_front['Price'].min()) * 0.05  # 5% margin
y_margin = (pareto_front['Walking Time to Centroid (minutes)'].max() - pareto_front['Walking Time to Centroid (minutes)'].min()) * 0.05  # 5% margin

# Update the layout to ensure the x-axis is numeric and increasing
fig.update_layout(
    xaxis=dict(type='linear', title='Price ($)', range=[pareto_front['Price'].min() - x_margin, pareto_front['Price'].max() + x_margin]),
    yaxis=dict(title='Walking Time (minutes)', range=[pareto_front['Walking Time to Centroid (minutes)'].min() - y_margin, pareto_front['Walking Time to Centroid (minutes)'].max() + y_margin]),
)

# Update the layout to make the graph more interactive
fig.update_traces(
    marker=dict(size=12, opacity=0.8, line=dict(width=2, color='DarkSlateGrey')),
    selector=dict(mode='markers')
)

# Show the plot
fig.show()

In [119]:
fig.write_html("pareto.html")